Distric of Columbia as DC

In [2]:
import pandas as pd
import numpy as np
import seaborn

In [3]:
def lower(x):
    return x.lower()
def change(x):
    if x in s2abr.keys():
        return s2abr[x] 
    else:
        return 'DC'

In [4]:
s2abr = pd.read_csv('data/state_to_abbr.csv')
s2abr['state'] = s2abr['state'].apply(lower)
s2abr = s2abr.set_index('state')
s2abr = s2abr.to_dict()['abbr']

READ AND CLEAN COMPLETE COUNTY LISTY

In [5]:
df_complete = pd.read_csv('data/raw/complete_county.csv')
print len(df_complete)

# change state name to abbreviation
df_complete['State'] = df_complete['State'].map(lambda x: x.replace('_', ''))
df_complete['State'] =  df_complete['State'].map(lower)
df_complete['State'] = df_complete['State'].map(change)
df_complete.head()

3143


,Sno,County,State,Rank,Population
0,1,Autauga County,AL,907,"55,514"
1,2,Baldwin County,AL,332,"190,790"
2,3,Barbour County,AL,1523,"27,201"
3,4,Bibb County,AL,1707,"22,597"
4,5,Blount County,AL,882,"57,826"


In [6]:
#remove alaska and hawaii and columbia
df_complete = df_complete[df_complete['State'] != 'HI']
df_complete = df_complete[df_complete['State'] != 'AK']
df_complete = df_complete[df_complete['State'] != 'DC']
len(df_complete)

3108

READ AND CLEAN DISTANCE

In [7]:
df_distance = pd.read_csv('data/county_distances.csv')
df_distance['County'] = df_distance['County'].apply(lower)
df_distance.head()

,County,State,Rank,Population,latitude,longitude,distance_100000,distance_300000,distance_500000,distance_1000000
0,autauga county,AL,907,55514.0,32.524961,-86.629505,55.468394,120.387228,120.387228,656.514912
1,baldwin county,AL,332,190790.0,30.607402,-87.718027,0.000000,36.620510,340.608832,601.274988
2,barbour county,AL,1523,27201.0,31.813697,-85.481079,74.924444,216.820215,234.120887,526.289453
3,bibb county,AL,1707,22597.0,33.032728,-87.080731,45.862509,65.199386,65.199386,681.752585
4,blount county,AL,882,57826.0,33.951944,-86.582932,40.717787,47.727390,47.727390,612.043635


READ AND CLEAN VOTES

In [8]:
df_votes_raw = pd.read_csv('data/normalised_votes.csv')
df_votes_raw = df_votes_raw.drop_duplicates()

In [9]:
df_votes_raw['State'] = df_votes_raw['State'].map(lambda x: x.replace('-', ' '))
df_votes_raw['State'] = df_votes_raw['State'].map(s2abr)
df_votes = df_votes_raw 

In [10]:
df_votes = df_votes[df_votes['State'] != 'HI']
df_votes = df_votes[df_votes['State'] != 'AK']
df_votes = df_votes[df_votes['State'] != 'DC']
df_votes = df_votes.reset_index(drop=True)


In [11]:
df_votes.head()

,State,County,Votes_Norm,Trump_Votes,Hillary_Votes,Independent_Votes,Total_Votes,Trump_Perc,Hillary_Perc,Independent_Perc
0,AL,Autauga County,0.494,18110.0,5908.0,643.0,24661.0,73.4,24.0,2.6
1,AL,Baldwin County,0.578,72780.0,18409.0,2901.0,94090.0,77.4,19.6,3.1
2,AL,Barbour County,0.056,5431.0,4848.0,111.0,10390.0,52.3,46.7,1.1
3,AL,Bibb County,0.556,6733.0,1874.0,141.0,8748.0,77.0,21.4,1.6
4,AL,Blount County,0.814,22808.0,2150.0,426.0,25384.0,89.9,8.5,1.7


BRIEF COMPARISON

In [12]:
df_votes.sort_values(by=['State', 'County'])
df_distance.sort_values(by=['State', 'County'])
df_complete.sort_values(by=['State', 'County'])


print '                 VOTES     DISTANCE   COMPLETE'
print 'number of counties', len(df_votes), len(df_distance), len(df_complete)
print 'number of states:', len(df_votes['State'].unique()), len(df_distance['State'].unique()), len(df_complete['State'].unique())

                 VOTES     DISTANCE   COMPLETE
number of counties 3107 3107 3108
number of states: 48 48 48


FIX DUPLICATES AND DELETE A COUNTY FROM VOTES (Broomfield County CO)

In [93]:
df_distance = df_distance.drop(df_distance.index[2780])
df_distance = df_distance.drop(df_distance.index[2809])
df_votes = df_votes.drop(df_votes.index[223])

In [95]:
df_distance = df_distance.reset_index(drop=True)
df_votes = df_votes.reset_index(drop=True)

In [21]:
df_votes[df_votes['State'] == 'CO']['County'][:20],  df_distance[df_distance['State'] == 'CO']['County'][:20]

(216          Adams County
 217        Alamosa County
 218       Arapahoe County
 219      Archuleta County
 220           Baca County
 221           Bent County
 222        Boulder County
 223     Broomfield County
 224        Chaffee County
 225       Cheyenne County
 226    Clear Creek County
 227        Conejos County
 228       Costilla County
 229        Crowley County
 230         Custer County
 231          Delta County
 232         Denver County
 233        Dolores County
 234        Douglas County
 235          Eagle County
 Name: County, dtype: object, 215                      adams county
 216                    alamosa county
 217                   arapahoe county
 218                  archuleta county
 219                       baca county
 220                       bent county
 221                    boulder county
 222    broomfield, city and county of
 223                    chaffee county
 224                   cheyenne county
 225                clear creek county
 2

In [13]:
# check whether states have the same number of counties
print '1 ------------'
states = df_votes['State'].unique()
for s in states:

        vote =  len(df_votes[df_votes['State'] == s])
        dist =  len(df_distance[df_distance['State'] == s])
        
        if vote != dist:
            print s
            print vote, dist
            print ""
            
print '2 -------------'
states = df_distance['State'].unique()
for s in states:

        vote =  len(df_votes[df_votes['State'] == s])
        dist =  len(df_distance[df_distance['State'] == s])
        
        if vote != dist:
            print s
            print vote, dist
            print ""

1 ------------
CO
64 63

VA
133 134

2 -------------
CO
64 63

VA
133 134



,State,County,Votes_Norm,Number_Votes
221,CO,Bent County,0.309,1892.0
222,CO,Boulder County,-0.487,176735.0
223,CO,Broomfield County,-0.141,37342.0
224,CO,Chaffee County,0.046,10999.0
225,CO,Cheyenne County,0.723,1076.0


In [14]:
df_votes.to_csv('data/final/votes_clean.csv')
df_distance.to_csv('data/final/distance_clean.csv')

DATA LOCATION

In [133]:
df_location = pd.read_csv('data/location.csv')
len(df_location)

3107

In [134]:
# check whether states have the same number of counties
print '1 ------------'
states = df_location['state'].unique()
for s in states:

        vote =  len(df_location[df_location['state'] == s])
        dist =  len(df_complete[df_complete['State'] == s])
        
        if vote != dist:
            print s
            print vote, dist
            print ""

1 ------------


In [135]:
df_location[df_location['state'] == 'CO']['county'][:20],  df_complete[df_complete['State'] == 'CO']['County'][:20]

(215          Adams
 216        Alamosa
 217       Arapahoe
 218      Archuleta
 219           Baca
 220           Bent
 221        Boulder
 222        Chaffee
 223       Cheyenne
 224    Clear Creek
 225        Conejos
 226       Costilla
 227        Crowley
 228         Custer
 229          Delta
 230         Denver
 231        Dolores
 232        Douglas
 233          Eagle
 234        El Paso
 Name: county, dtype: object, 244                      Adams County
 245                    Alamosa County
 246                   Arapahoe County
 247                  Archuleta County
 248                       Baca County
 249                       Bent County
 250                    Boulder County
 251    Broomfield, City and County of
 252                    Chaffee County
 253                   Cheyenne County
 254                Clear Creek County
 255                    Conejos County
 256                   Costilla County
 257                    Crowley County
 258                     

In [131]:
df_complete = df_complete.drop(df_complete.index[251])

In [132]:
df_complete.to_csv('data/*complete_edited.csv')